In [1]:
#!pip install dash==1.19.0  
#!pip install jupyter_dash 
#!pip install --upgrade plotly

Whenever you make changes to your code and want to rerun it  close the running instance, shutdown the kernel and restart the kernel and rerun the cells.

In [2]:
#Skeleton Code copied from:
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py

In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import plotly.graph_objects as go

In [4]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'

In [5]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [6]:
# For marks slider Task 3
mark = {0:'0'}
for num in range(1000,10001,1000):
    mark[num] = str(num)

In [7]:
# For labels
launchsites = spacex_df['Launch Site'].unique()
launchsites

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [8]:
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site_dropdown',
                                            options=[
                                                {'label':'All Sites','value':'ALL'},
                                                {'label':launchsites[0],'value':launchsites[0]},
                                                {'label':launchsites[1],'value':launchsites[1]},
                                                {'label':launchsites[2],'value':launchsites[2]},
                                                {'label':launchsites[3],'value':launchsites[3]}
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site",
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # return the outcomes piechart for a selected site
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='pie-chart')),
                                html.Br(),
                                
                                # TASK 3: Add a slider to select payload range
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='payload_slider',
                                               min = 0,
                                                max =10000,
                                                step=1000,
                                                marks=mark,
                                                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='pie-chart', component_property='figure'),
              [Input(component_id='site_dropdown', component_property='value'),
               Input(component_id='payload_slider', component_property='value')])

def get_pie_chart(site_dropdown,payload_slider):
    low,high=payload_slider
    
    ranged_df=spacex_df[(spacex_df['Payload Mass (kg)'] >= low)&
                       (spacex_df['Payload Mass (kg)'] <= high)]
    
    filtered_df = ranged_df[ranged_df['Launch Site']==site_dropdown]
    #filtered_df = spacex_df[spacex_df['Launch Site']==site_dropdown]
    value = filtered_df['class'].value_counts()
    name = names=filtered_df['class'].unique()
    if site_dropdown == 'ALL':
        fig = px.pie(ranged_df, values='class', 
                     names='Launch Site', 
                     title='Success Rate of Launch Sites')
        return fig
    else:
        fig = px.pie(filtered_df, values=value, 
                     names=name, 
                     title='Launch Site Outcome')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='scatter-chart', component_property='figure'),
              [Input(component_id='site_dropdown', component_property='value'),
               Input(component_id='payload_slider', component_property='value')])

def get_scatter_plot(site_dropdown,payload_slider):    
    low,high=payload_slider
    
    ranged_df=spacex_df[(spacex_df['Payload Mass (kg)'] >= low)&
                       (spacex_df['Payload Mass (kg)'] <= high)]
    
    filtered_df = ranged_df[ranged_df['Launch Site']==site_dropdown]
    
    if site_dropdown == 'ALL':
        fig = px.scatter(data_frame=ranged_df,x='Payload Mass (kg)',y='class',
                         color='Booster Version Category',
                         labels= 'Booster Version Category')
        return fig
    else:
        fig = px.scatter(data_frame=filtered_df,x='Payload Mass (kg)',y='class',
                         color='Booster Version Category',
                         labels= 'Booster Version Category')
        return fig


# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
                   dev_tools_hot_reload =True, threaded=True)
    
print('Successfully Run')

Successfully Run


To see in browser: http://localhost:8090